In [1]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

Using TensorFlow backend.


In [22]:
img_width, img_height = 16, 16

nb_epoch = 50
batch_size = 32

In [23]:
base_dir = '/home/niaki/Code/ImageNet/tiny-imagenet-200'

train_data_dir      = base_dir + '/tiny_train16'
validation_data_dir = base_dir + '/tiny_validation16'
test_data_dir       = base_dir + '/tiny_test16'

train_descrs_dir      = base_dir + '/tiny_sifts/tiny_train16'
validation_descrs_dir = base_dir + '/tiny_sifts/tiny_validation16'
test_descrs_dir       = base_dir + '/tiny_sifts/tiny_test16'

In [24]:
def loading_data(data_dir):
    files = listdir(data_dir + '/class0')
    files.sort()

    images = []

    for file in files:
        image = imageio.imread(data_dir + '/class0/' + file)
    #     image = np.expand_dims(image, axis=0)
        images.append(image)

    images = np.array(images)
    images = images.astype(np.float64) / 255
    print(images.shape)
    return images

In [25]:
x_train = loading_data(train_data_dir)
x_validation = loading_data(validation_data_dir)

(157086, 16, 16, 3)
(3932, 16, 16, 3)


In [26]:
def loading_descrs(descrs_dir):
    files = listdir(descrs_dir + '/class0')
    files.sort()

    descrs = []

    for file in files:
        descr = np.load(descrs_dir + '/class0/' + file)
        descrs.append(descr)

    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print(descrs.shape)
    return descrs

In [27]:
y_train = loading_descrs(train_descrs_dir)
y_validation = loading_descrs(validation_descrs_dir)

(157086, 128)
(3932, 128)


In [113]:
# to be deleted from here...

In [83]:
temp_y_prime

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [81]:
temp_y.reshape((1,128))

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,  39., 137., 106.,  43.,
         47.,  71., 119.,  85.,  53., 137., 102.,  23.,  22.,  34., 107.,
        137.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,  32.,  89.,  52.,  67.,  91.,
        113., 137., 137.,  47., 116.,  50.,  31.,  41.,  54., 137., 137.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)

In [86]:
def customLoss_check(y_true, y_pred):
    return np.sum(y_true - y_pred)

In [88]:
def customLoss(y_true, y_pred):
    return K.sum(y_true - y_pred)
#     return K.sum(K.log(y_true) - K.log(y_pred))

In [89]:
customLoss(temp_y.reshape((1,128)), temp_y_prime)

<tf.Tensor 'Sum_1:0' shape=() dtype=float32>

In [87]:
customLoss_check(temp_y.reshape((1,128)), temp_y_prime)

2590.0

In [94]:
customLoss(3, 15)

<tf.Tensor 'Sum_6:0' shape=() dtype=int32>

In [100]:
K.sum((temp_y.reshape((1,128)), temp_y_prime))

ValueError: Argument must be a dense tensor: (array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,  39., 137., 106.,  43.,
         47.,  71., 119.,  85.,  53., 137., 102.,  23.,  22.,  34., 107.,
        137.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,  32.,  89.,  52.,  67.,  91.,
        113., 137., 137.,  47., 116.,  50.,  31.,  41.,  54., 137., 137.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,


In [102]:
a = np.array([1,2,3], dtype=np.float32).reshape((1,3))

In [104]:
b = np.array([100,20,300], dtype=np.float32).reshape((1,3))

In [109]:
ab = np.zeros((2,3), dtype=np.float32)
ab[0] = a
ab[1] = b

In [111]:
K.sum(ab)

<tf.Tensor 'Sum_9:0' shape=() dtype=float32>

<module 'keras.backend' from '/scratch/tensorflow/lib/python3.6/site-packages/keras/backend/__init__.py'>

In [ ]:
temp1 = np.array()

In [114]:
# ... until here!

In [7]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [8]:
input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(128, activation="sigmoid")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', metrics=['accuracy'], loss=MSLE_plus_plus)
#next up: encoder.compile(optimizer='sgd', metrics=['categorical_accuracy'], loss='categorical_crossentropy')


encoder.summary()

W1118 16:30:13.426448 139665879115584 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1118 16:30:13.446276 139665879115584 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1118 16:30:13.451584 139665879115584 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1118 16:30:13.467727 139665879115584 deprecation_wrapper.py:119] From /scratch/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1118 16:30:13.5008

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 16, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 32)          0         
__________

In [28]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [29]:
model_version = '1.0.0.6_encoder_lastsigmoid_mslepp_adadelta_metricsaccuracy_trainedmore'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, 32),
                steps_per_epoch=157086,
                epochs=70,
                validation_data=fixed_generator(x_validation, y_validation, 32),
                validation_steps=3932,
                callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

# import keras.losses
# keras.losses.MSLE_plus_plus = MSLE_plus_plus
# encoder = load_model(base_dir + '/ae' + model_version + '.h5')

Epoch 1/70
157086/157086 [==============================] - 813s 5ms/step - loss: 0.0065 - acc: 0.1210 - val_loss: 0.0067 - val_acc: 0.1499
Epoch 2/70
157086/157086 [==============================] - 802s 5ms/step - loss: 0.0065 - acc: 0.1203 - val_loss: 0.0064 - val_acc: 0.1342
Epoch 3/70
157086/157086 [==============================] - 796s 5ms/step - loss: 0.0065 - acc: 0.1183 - val_loss: 0.0065 - val_acc: 0.1161
Epoch 4/70
157086/157086 [==============================] - 796s 5ms/step - loss: 0.0065 - acc: 0.1172 - val_loss: 0.0065 - val_acc: 0.1066
Epoch 5/70
157086/157086 [==============================] - 787s 5ms/step - loss: 0.0065 - acc: 0.1175 - val_loss: 0.0065 - val_acc: 0.1322
Epoch 6/70
157086/157086 [==============================] - 786s 5ms/step - loss: 0.0065 - acc: 0.1181 - val_loss: 0.0080 - val_acc: 0.0935
Epoch 7/70
157086/157086 [==============================] - 785s 5ms/step - loss: 0.0065 - acc: 0.1190 - val_loss: 0.0065 - val_acc: 0.1202
Epoch 8/70
157086/15

157086/157086 [==============================] - 785s 5ms/step - loss: 0.0065 - acc: 0.1203 - val_loss: 0.0064 - val_acc: 0.1080
Epoch 60/70
157086/157086 [==============================] - 785s 5ms/step - loss: 0.0065 - acc: 0.1201 - val_loss: 0.0064 - val_acc: 0.1280
Epoch 61/70
157086/157086 [==============================] - 786s 5ms/step - loss: 0.0065 - acc: 0.1194 - val_loss: 0.0067 - val_acc: 0.0982
Epoch 62/70
157086/157086 [==============================] - 786s 5ms/step - loss: 0.0065 - acc: 0.1192 - val_loss: 0.0064 - val_acc: 0.1265
Epoch 63/70
157086/157086 [==============================] - 786s 5ms/step - loss: 0.0065 - acc: 0.1190 - val_loss: 0.0064 - val_acc: 0.1062
Epoch 64/70
157086/157086 [==============================] - 787s 5ms/step - loss: 0.0065 - acc: 0.1188 - val_loss: 0.0064 - val_acc: 0.1348
Epoch 65/70
157086/157086 [==============================] - 787s 5ms/step - loss: 0.0065 - acc: 0.1188 - val_loss: 0.0065 - val_acc: 0.1194
Epoch 66/70
157086/157086

In [13]:
encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_test16/class0/patch000026.png").reshape(1,16,16,3))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [45]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000012.png").reshape(1,16,16,3))

In [46]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000012.npy")

In [47]:
print(np.corrcoef(temp_y, temp_y_prime[0])[1,0], "\n")
for i in range(temp_y_prime.shape[1]):
    print('{:>12f}{:>12f}'.format(temp_y[i], temp_y_prime[0, i]))      
#     print(temp_y_prime[0, i], temp_y[i])

0.6393183138370047 

    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0